In [1]:
#Q1
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

training_epochs = 15 # training 반복 횟수
batch_size = 100 # 100장씩

root = './data'
mnist_train = dset.MNIST (root=root, train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dset.MNIST (root=root, train=False, transform=transforms.ToTensor(), download=True)

# data loader를 직접 구현해보자.
train_loader = DataLoader(dataset= mnist_train, batch_size = batch_size, shuffle = True, drop_last=True)
test_loader = DataLoader(dataset= mnist_test, batch_size = batch_size, shuffle= True, drop_last= True)

ModuleNotFoundError: No module named 'torch'

In [2]:
#Q2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
linear = torch.nn.Linear(784, 10, bias=True).to(device) #입력 784 아웃풋 10
# weight init
torch.nn.init.normal(linear.weight)

C:\Users\tomat\AppData\Local\Temp\ipykernel_6016\1333838360.py:5: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(linear.weight)


Parameter containing:
tensor([[ 0.3987, -0.7948, -0.6429,  ...,  0.4226, -0.4063, -0.2178],
        [ 0.0977,  0.3655,  0.2336,  ..., -0.3699, -1.0575,  0.9655],
        [ 0.9843,  0.8237, -1.3482,  ...,  1.1786, -1.4493, -0.7871],
        ...,
        [-0.9830,  0.5927,  0.1356,  ...,  0.0431, -0.6313,  1.5476],
        [ 2.9886, -1.0037,  0.7983,  ..., -0.7353,  0.3329,  0.5755],
        [-0.8757,  0.1991, -1.2163,  ...,  0.4949, -0.9354, -0.2306]],
       device='cuda:0', requires_grad=True)

In [3]:
#Q3
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

In [6]:
#Q4
for epoch in range(training_epochs):
    for i, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28*28).to(device)

        outputs =linear(imgs)
        loss =criterion(outputs,labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _,argmax = torch.max(outputs, 1)
        accuracy = (labels == argmax).float().mean()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {: .4f}, Accuracy: {: .2f}%'.format(
            epoch+1, training_epochs, i+1, len(train_loader), loss.item(), accuracy.item()* 100))

    print(f'Epoch [{epoch+1}/{training_epochs}]  Everage Accuracy: { accuracy.mean()*100:.2f}%')

Epoch [1/15], Step [100/600], Loss:  0.3507, Accuracy:  91.00%
Epoch [1/15], Step [200/600], Loss:  0.2653, Accuracy:  93.00%
Epoch [1/15], Step [300/600], Loss:  0.2913, Accuracy:  93.00%
Epoch [1/15], Step [400/600], Loss:  0.2185, Accuracy:  91.00%
Epoch [1/15], Step [500/600], Loss:  0.5914, Accuracy:  91.00%
Epoch [1/15], Step [600/600], Loss:  0.3145, Accuracy:  92.00%
Epoch [1/15]  Everage Accuracy: 92.00%
Epoch [2/15], Step [100/600], Loss:  0.3493, Accuracy:  89.00%
Epoch [2/15], Step [200/600], Loss:  0.3063, Accuracy:  90.00%
Epoch [2/15], Step [300/600], Loss:  0.2823, Accuracy:  94.00%
Epoch [2/15], Step [400/600], Loss:  0.4227, Accuracy:  89.00%
Epoch [2/15], Step [500/600], Loss:  0.4479, Accuracy:  90.00%
Epoch [2/15], Step [600/600], Loss:  0.4002, Accuracy:  85.00%
Epoch [2/15]  Everage Accuracy: 85.00%
Epoch [3/15], Step [100/600], Loss:  0.3118, Accuracy:  93.00%
Epoch [3/15], Step [200/600], Loss:  0.2045, Accuracy:  98.00%
Epoch [3/15], Step [300/600], Loss:  0.5

KeyboardInterrupt: 

In [27]:
##Q5
linear.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (imgs, labels) in enumerate(test_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1, 28 * 28)

        outputs = linear(imgs)

        _, argmax = torch.max(outputs, 1) # max()를 통해 최종 출력이 가장 높은 class 선택
        total += imgs.size(0)
        correct += (labels == argmax). sum().item()

    print('Test accuracy for {} images: {: .2f}%'.format(total, correct / total * 100))

Test accuracy for 10000 images:  90.43%
